In [4]:
from Models.PPO.PPO_Agent import PPO_Agent
import torch
import slimevolleygym
from slimevolleygym import BaselinePolicy
from utils import convert_to_vector, convert_to_value
import types

In [5]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')

# Print the device as a check
print("Device used: ", DEVICE)

Device used:  cuda:0


In [6]:
env = slimevolleygym.SlimeVolleyEnv()

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

agent1 = PPO_Agent(12, 6, DEVICE, mlp_layers=[256, 128])
agent1.load_models("Logging/PPO-BASELINE/20240408-190946-lr-0.001-entcoef-0-mlp-256-128-kl-0.02", 1, 12337140)

agent2 = BaselinePolicy()
def select_action(self, state, greedy=False):
    action = self.predict(state)
    return convert_to_value(action), None

# def select_action(self, state, greedy=False):
#     action = env.action_space.sample()
#     return convert_to_vector(action), None

def evaluation_mode(self):
    pass

agent2.select_action = types.MethodType(select_action, agent2)
agent2.evaluation_mode = types.MethodType(evaluation_mode, agent2)

In [7]:
# Set the agents to evaluation mode
agent1.evaluation_mode()
agent2.evaluation_mode()

# Run num_eval_episodes episodes and calculate the total return
total_return = 0
for _ in range(10):

    state1 = env.reset()
    state2 = state1
    done = False
    while not done:

        # Render the environment
        env.render(mode="human")
        
        with torch.no_grad():

            # Select the actions for each agent
            action1, _ = agent1.select_action(state1, greedy=True)
            action2, _ = agent2.select_action(state2, greedy=True)
        
        # Step the environment forward
        next_state1, reward, done, info = env.step(convert_to_vector(action1), otherAction=convert_to_vector(action2))
        next_state2 = info['otherObs']
        
        # Add the individual agents' rewards to the total returns (Since they're the same for both agents)
        total_return += reward

        # Update the states
        state1 = next_state1
        state2 = next_state2

KeyboardInterrupt: 